In [1]:
import pandas as pd

In [3]:
fund_stat = pd.read_excel("fund_name_risk.xlsx")

In [4]:
fund_stat

,name,code,sharp_ratio,std
0,野村優質基金-累積類型新臺幣計價,ACIC01,0.420179,0.167070
1,野村 e科技基金,ACIC06,0.353621,0.213087
2,野村中小基金-累積類型,ACIC08,0.421352,0.170263
3,野村全球高股息基金-累積型新臺幣計價,ACIC10,0.309774,0.112105
4,野村精選貨幣市場基金,ACKH22,-55.294615,0.000149
5,野村全球生技醫療基金,ACIC19,0.407442,0.169309
6,野村新興傘型之大俄羅斯基金,NaN,0.000000,0.000000
7,野村中國機會基金,ACIC26,0.308278,0.180038
8,野村全球美元投資級公司債基金-累積型,NaN,0.000000,0.000000
9,野村巴西證券投資信託基金,NaN,0.000000,0.000000


In [5]:
invest = pd.read_excel("invest_data.xlsx")